#Postgresql


In [ ]:
import psycopg2
from psycopg2 import sql
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, UniqueConstraint, ForeignKey, BIGINT, Text
from sqlalchemy.orm import relationship, declarative_base

In [ ]:
# Create the engine
DATABASE_URL = 'postgresql+psycopg2://u381r20ceebmb7:p2c1b3eb128bb09f92c43d005d55f54c36a4a0e5bd110945652252726dfdb6068@c3gtj1dt5vh48j.cluster-czrs8kj4isg7.us-east-1.rds.amazonaws.com:5432/d77oud95l1v4g6'
engine = create_engine(DATABASE_URL)
metadata = MetaData()

### Create Company Information Table

Create Company Metrics Table (Do Not Run)

In [ ]:
Base = declarative_base()

class CompanyInfo(Base):
  __tablename__ = 'company_information'

  Ticker = Column(String(255))
  CompanyName = Column(String(255))
  Sector = Column(String(255))
  CompanyID = Column(Integer, primary_key=True, autoincrement=False)

  metrics = relationship('company_metrics', back_populates='company')
  ratios = relationship('company_ratios', back_populates='company')
  summaries = relationship('company_ten_k_summaries', back_populates)

class CompanyMetrics(Base):
  __tablename__ = 'company_metrics'

  MetricID = Column(Integer, primary_key=True, autoincrement=True)
  CompanyID = Column(Integer, ForeignKey('company_information.CompanyID'), nullable=False)
  Year = Column(Integer, nullable=False)
  Quarter = Column(Integer, nullable=False)
  Form_Type = Column(String(255), nullable=False)

  Adjusted_Price_Per_Share = Column(Float, nullable=False)
  Basic_EPS = Column(Float, nullable=False)
  Total_Common_Shares_Outstanding = Column(BIGINT, nullable=False)
  Total_Assets = Column(BIGINT, nullable=False)
  Total_Liabilities = Column(BIGINT, nullable=False)
  Dividends = Column(BIGINT, nullable=False)
  Revenue = Column(BIGINT, nullable=False)
  Net_Income = Column(BIGINT, nullable=False)
  Capital_Expenditure = Column(BIGINT, nullable=False)
  Operating_Cash_Flow = Column(BIGINT, nullable=False)

  __table_args__ = (
    UniqueConstraint('CompanyID', 'Year', 'Quarter', name='company_metrics_constraint'),
  )

  company = relationship('company_information', back_populates='metrics')

class CompanyRatios(Base):
  __tablename__ = 'company_ratios'

  RatioID = Column(Integer, primary_key=True, autoincrement=True)
  CompanyID = Column(Integer, ForeignKey('company_information.CompanyID'), nullable=False)
  Year = Column(Integer, nullable=False)
  Quarter = Column(Integer, nullable=False)

  Price_To_Earnings_Ratio = Column(Float, nullable=False)
  Price_To_Book_Ratio = Column(Float, nullable=False)
  Debt_To_Equity_Ratio = Column(Float, nullable=False)
  Dividend_Yield = Column(Float, nullable=False)
  Free_Cash_Flow = Column(BIGINT, nullable=False)

  __table_args__ = (
    UniqueConstraint('CompanyID', 'Year', 'Quarter', name='company_ratios_constraint'),
  )



Base.metadata.create_all(engine, checkfirst=True)

print("Tables created successfully (assuming they don't already exist)")

Tables created successfully (assuming they don't already exist)


In [ ]:
class CompanyTenKSummaries(Base):
    __tablename__ = 'company_ten_k_summaries'

    SummaryID = Column(Integer, primary_key=True, autoincrement=True)
    CompanyID = Column(Integer, ForeignKey('company_information.CompanyID'), nullable=False)
    Year = Column(Integer, nullable=False)
    SummaryText = Column(Text, nullable=False)

    __table_args__ = (
      UniqueConstraint('CompanyID', 'Year', name='company_ten_k_summaries_constraint'),
    )

    company = relationship('company_information', back_populates='ten_k_summaries')

# Update the relationship in CompanyInfo
CompanyInfo.ten_k_summaries = relationship('CompanyTenKSummaries', back_populates='company')
CompanyRatios.company = relationship('CompanyInfo', back_populates='ratios')

# Create the table in the database
Base.metadata.create_all(engine, checkfirst=True)

print("CompanyTenKSummaries table created successfully (if it doesn't already exist)")

CompanyTenKSummaries table created successfully (if it doesn't already exist)


In [ ]:
table = Table("company_ten_k_summaries", metadata)

table.drop(engine, checkfirst=True)  # Drops the table if it exists
print("Table company_ten_k_summaries dropped using ORM.")

In [ ]:
sp500_SEC.to_sql('company_information', con=engine, if_exists='append', index=False)

500

In [ ]:
table_name = "company_ten_k_summaries"
query = f"SELECT * FROM {table_name}"
df = pd.read_sql(query, con=engine)

In [ ]:
print(df)

   SummaryID  CompanyID  Year  \
0          9       2488  2024   
1         10     882095  2024   
2         11      70858  2024   
3         12    1045810  2024   
4         13      59478  2024   
5         14      12927  2024   
6         15    1551152  2024   
7         16     753308  2024   

                                         SummaryText  
0  Advanced Micro Devices, Inc. (AMD) provided a ...  
1  The company is facing various risks that could...  
2  Bank of America's approach to managing its hum...  
3  NVIDIA's financial performance for fiscal year...  
4  The company's discussion and analysis of resul...  
5  Boeing's Financial Summary for 2023, highlight...  
6  AbbVie Inc., a global biopharmaceutical compan...  
7  NextEra Energy's (NEE) operating performance f...  
